## Assignment 2

In [4]:
# Import Packages
import pandas as pd
from cassandra.cluster import Cluster

In [5]:
# Creating connection to Cassendra Cluster
clstr = Cluster()
session = clstr.connect()

In [6]:
# Check for keyspace and drop if it exists
session.execute("DROP KEYSPACE IF EXISTS m14")

In [7]:
# View Keyspaces
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [8]:
# Create Keyspace m14
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [9]:
# View keyspaces
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [10]:
# Create table
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory ( 
    sku VARCHAR, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT, 
    PRIMARY KEY(sku)
);
""")

In [11]:
### Load Data
df = pd.read_csv('data.csv')
df

,sku,name,description,warehouse_num
0,'AAA-1','Laptop','Portable computer for work and entertainment.',1
1,'AAA-2','Guitar','Musical instrument for playing melodies and c...,3
2,'AAA-3','Clock','Timekeeping device for tracking hours and min...,1
3,'BBB-1','Bag','Carries and stores personal items and belongi...,2
4,'BBB-2','Pen','Writing instrument for ink-based writing.',3
5,'BBB-3','Bottle','Container for holding liquids or beverages.',3
6,'CCC-1','Book','Printed content for reading and information.',1
7,'CCC-2','Chair','Seating furniture for comfort and support.',2


In [12]:
# Iterate throught the dataframe and print it.
for index, row in df.iterrows():
    print(f"SKU = {row.sku}, name ={row.name}, description = {row.description}, warehouse_num = {row.warehouse_num}")

SKU = 'AAA-1', name =0, description = 'Portable computer for work and entertainment.', warehouse_num = 1
SKU = 'AAA-2', name =1, description = 'Musical instrument for playing melodies and chords.', warehouse_num = 3
SKU = 'AAA-3', name =2, description = 'Timekeeping device for tracking hours and minutes.', warehouse_num = 1
SKU = 'BBB-1', name =3, description = 'Carries and stores personal items and belongings.', warehouse_num = 2
SKU = 'BBB-2', name =4, description = 'Writing instrument for ink-based writing.', warehouse_num = 3
SKU = 'BBB-3', name =5, description = 'Container for holding liquids or beverages.', warehouse_num = 3
SKU = 'CCC-1', name =6, description = 'Printed content for reading and information.', warehouse_num = 1
SKU = 'CCC-2', name =7, description = 'Seating furniture for comfort and support.', warehouse_num = 2


In [13]:
# Print the values that are being inserted into the table
for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
# Insert the data into the table
    session.execute(f"""
        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )


        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('AAA-1', 'Laptop', 'Portable computer for work and entertainment.', 1);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('AAA-2', 'Guitar', 'Musical instrument for playing melodies and chords.', 3);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('AAA-3', 'Clock', 'Timekeeping device for tracking hours and minutes.', 1);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('BBB-1', 'Bag', 'Carries and stores personal items and belongings.', 2);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('BBB-2', 'Pen', 'Writing instrument for ink-based writing.', 3);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('BBB-3', 'Bottl

In [14]:
#Retrieving and Printing Inventory Data from a Database
rows = session.execute("select (sku, name, description, warehouse_num) from m14.inventory")
for row in rows:
    print(f"SKU={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")

SKU=AAA-2, name=Guitar, description=Musical instrument for playing melodies and chords., warehouse_num=3
SKU=AAA-3, name=Clock, description=Timekeeping device for tracking hours and minutes., warehouse_num=1
SKU=CCC-2, name=Chair, description=Seating furniture for comfort and support., warehouse_num=2
SKU=BBB-2, name=Pen, description=Writing instrument for ink-based writing., warehouse_num=3
SKU=BBB-1, name=Bag, description=Carries and stores personal items and belongings., warehouse_num=2
SKU=CCC-1, name=Book, description=Printed content for reading and information., warehouse_num=1
SKU=BBB-3, name=Bottle, description=Container for holding liquids or beverages., warehouse_num=3
SKU=AAA-1, name=Laptop, description=Portable computer for work and entertainment., warehouse_num=1


In [15]:
# Create Index on warehouse_num
session.execute("""CREATE INDEX IF NOT EXISTS warehouse_num ON m14.inventory(warehouse_num);""")

In [55]:
# Search products at a given location
rows = session.execute("select name from m14.inventory where warehouse_num = 1 ")
for row in rows:
    print(f"{row[0]}")

Clock
Book
Laptop
